In [8]:
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWgLULpeOS6pjfyyQYWLuyCMWG4sU9F8alB2Becb88cpUWPf023DUM4

Successfully saved authorization token.


In [9]:
import os
import json
import earthpy as et
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
import IPython.display as disp
%matplotlib inline

# change the global options that Geopandas inherits from
pd.set_option('display.max_columns',None)

In [10]:
# Change directory to landslide-detect data path
data_path = os.path.join(et.io.HOME, "earth-analytics", "landslide-detect")
if os.path.exists(data_path):
    os.chdir(data_path)
else:
    os.makedirs(data_path)
    print('The new directory is created!')
    os.chdir(data_path)

print('Current working directory is set to: ', os.getcwd())

Current working directory is set to:  /Users/leahmanak/earth-analytics/landslide-detect


In [11]:
def create_dataframe(data_file):
    """
    Reads a csv file into a GeoPandas DataFrame. Sets the index to the 'slide.id'
    and sorts the geoDataFrame by the new index. Converts the date+time column to
    a DateTime object and drops the time. Converts the reported geocoordinates to
    a geoPandas geometry point.

    Parameters
    ----------
    data_file : str

    Returns
    -------
    dataframe
        sorted by index
    """

    gdf = gpd.read_file(data_file, index_col='slide.id').sort_values(by='slide.id')
    gdf = gdf.set_index('slide.id')
    # Convert to DatTime object, drop time
#     gdf['slide.date'] = pd.to_datetime(gdf['slide.date'])
#     gdf['slide.date'] = gdf['slide.date'].dt.date
    # Convert reported geocoordinates to geometery
    gdf[['lon', 'lat']] = gdf[['lon', 'lat']].astype(float)
    geometry = [Point(xy) for xy in zip(gdf.lon, gdf.lat)]
    gdf = gdf.drop(['lon', 'lat'], axis=1)
    gdf = GeoDataFrame(gdf, crs="EPSG:4326", geometry=geometry)

    return gdf

In [17]:
def image_search(gdf):
    """
    Iterates through a geoDataFrame and searches Google Earth Engine
    based on certain parameters within the geoDataFrame. Buffers central
    geometery search point by 1000 meters, event date by 180 days reciprocal. 

    
    Parameters
    ----------
    data_file : geodataframe

    Returns
    -------
    dataframe
        appended with imagery timestamplist
    
    """

    # Set empty list
    results = []
    
    BASE_DATE = ee.Date(gdf['slide.date'])
    
    # Data search in Google Earth Engine
    im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
               .filterBounds(gdf['geometry'])
               .filter(ee.Geometry.Point.buffer({'distance': 1000}))
               .filterDate(BASE_DATE, 
                           BASE_DATE.advance(-180, 'days'), 
                           BASE_DATE.advance(180, 'days'))
#                .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
               .map(lambda img: img.set('date', ee.Date(img.date()).format('YYYYMMdd')))
               .sort('date'))
    timestamplist = (im_coll.aggregate_array('date')
                            .map(lambda d: ee.String('T').cat(ee.String(d)))
                            .getInfo())
    
    # Add new column to geopandas DataFrame for avaiable data
    for value in timestamplist:
        if value == True:
            results.append(timestamplist)
        else:
            results.append('No imagery available')

    return gdf



NameError: name 'gdf' is not defined

In [13]:
# for id, location, point in zip(landslide_df.index,
#                                landslide_df['location'],
#                                landslide_df['point'],
#                                ):
#     print(id, location, point)

In [14]:
for row in landslide_gdf.iterrows():
    print(landslide_gdf['geometry'])

NameError: name 'landslide_gdf' is not defined

In [15]:
landslide_gdf = create_dataframe('landslides.verified.copy.csv')
landslide_gdf.head()

DriverError: landslides.verified.copy.csv: No such file or directory